API extraction of Yelp Fusion API (data from Santa Barbara, Goleta, Isla Vista) are stored in .csv files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install flask
# !pip install requests

In [ ]:
from flask import Flask, render_template, request  # Import Flask and related modules for web handling
import requests  # Import requests library to make HTTP requests
import http.client
from google.colab import files
import json
import pandas as pd
import numpy as np

In [ ]:
# Yelp API credentials
client_id = 'LcBbj5dNVQzQ4uc2vEb95g'
api_key = 'EXkqJVsBaTi_X3W99nzVXhCSEYfj8ahAxYaGRKQe5EIrJpgqbWHHMJ7Pg1T0RxWhimJA6eGiwzZuD1s-0MsQ7sFB6doDa4hW8ZW01680fxU172UkM4ZDFTWyF-RUZ3Yx'

# Yelp API endpoint for searching businesses
url = 'https://api.yelp.com/v3/businesses/search'

In [ ]:
# function to return a dataframe of 50 restaurants in area
def get_restaurants(city, term='restaurants', limit = 50):
    # Construct the Yelp API URL
    url = 'https://api.yelp.com/v3/businesses/search'

    # Set up the headers for authentication
    headers = {
        'Authorization': f'Bearer {api_key}'
    }

    # Define the parameters for the request
    params = {
        'location': city,
        'term': term,
        'limit': limit
    }

    # Send a GET request to the Yelp API
    response = requests.get(url, headers=headers, params=params)

    # Check if the response was successful
    if response.status_code == 200:
        # Return the JSON data AS A DATAFRAME if the request was successful
        response_data = response.json()

        restaurants_df = pd.DataFrame()
        for i in range(len(response_data['businesses'])):
          # input restaurant information
          restaurants_df.loc[i, 'Business_ID'] = response_data['businesses'][i].get('id', None)
          restaurants_df.loc[i, 'Name'] = response_data['businesses'][i].get('name', None)
          restaurants_df.loc[i, 'NumReviews'] = response_data['businesses'][i].get('review_count', None)
          restaurants_df.loc[i, 'Rating'] = response_data['businesses'][i].get('rating', None)

          # input information from categories tab
          categories = response_data['businesses'][i].get('categories', [])
          if len(categories) > 0:
            restaurants_df.loc[i, 'Cuisine'] = categories[0]['title']
          if len(categories) > 1:
            restaurants_df.loc[i, 'Cuisine1'] = categories[1]['title']
          if len(categories) > 2:
            restaurants_df.loc[i, 'Cuisine2'] = categories[2]['title']

          restaurants_df.loc[i, 'PriceRange'] = response_data['businesses'][i].get('price', None)
          restaurants_df.loc[i, 'Takeout'] = ', '.join(response_data['businesses'][i].get('transactions', [])) # rewrite list as string
          restaurants_df.loc[i, 'Waitlist_Needed'] = response_data['businesses'][i].get('attributes', {}).get('waitlist_reservation', None)

          # business hours (check if any open hours)
          business_hours = response_data['businesses'][i].get('business_hours', [])
          if business_hours:
              open_hours = business_hours[0].get('open', [])
          else:
              open_hours = []
          days = ['Su', 'M', 'T', 'W', 'R', 'F', 'Sa']

          for j in range(7):
              if j < len(open_hours):
                  restaurants_df.loc[i, f'{days[j]}_start'] = open_hours[j].get('start', None)
                  restaurants_df.loc[i, f'{days[j]}_end'] = open_hours[j].get('end', None)
              else:
                  restaurants_df.loc[i, f'{days[j]}_start'] = None
                  restaurants_df.loc[i, f'{days[j]}_end'] = None

          restaurants_df.loc[i, 'Latitude'] = response_data['businesses'][i].get('coordinates', {}).get('latitude', None)
          restaurants_df.loc[i, 'Longitude'] = response_data['businesses'][i].get('coordinates', {}).get('longitude', None)
          restaurants_df.loc[i, 'Address'] = response_data['businesses'][i].get('location', {}).get('address1', None)
          restaurants_df.loc[i, 'City'] = response_data['businesses'][i].get('location', {}).get('city', None)
          restaurants_df.loc[i, 'State'] = response_data['businesses'][i].get('location', {}).get('state', None)
          restaurants_df.loc[i, 'Zip'] = response_data['businesses'][i].get('location', {}).get('zip_code', None)
          restaurants_df.loc[i, 'Menu'] = response_data['businesses'][i].get('attributes', {}).get('menu_url', None)

        return restaurants_df
    else:
        # Return None if there was an error
        return None

In [ ]:
sb_restaurants = get_restaurants('Santa Barbara')

In [ ]:
iv_restaurants = get_restaurants('Isla Vista')

In [ ]:
sb_restaurants.keys()

dict_keys(['businesses', 'total', 'region'])

In [ ]:
iv_restaurants.to_csv('iv_restaurants.csv', index=False)
files.download('iv_restaurants.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Code below this is to test if we can extract REVIEWS (if necessary)

In [ ]:
def get_reviews(business_id):
    url = f'https://api.yelp.com/v3/businesses/{business_id}/reviews'

    headers = {
        'Authorization': f'Bearer {api_key}'
    }

    conn = http.client.HTTPSConnection("api.yelp.com")
    conn.request("GET", f"/v3/businesses/{business_id}/reviews", headers=headers)
    res = conn.getresponse()
    data = res.read()
    print(data.decode("utf-8"))

#    response = requests.get(url, headers=headers)

#    if response.status_code == 200:
#        return response.json().get('reviews', [])
#    else:
#        print(f"Error fetching reviews for {business_id}: {response.status_code} - {response.text}")
#        return []

In [ ]:
url = "https://api.yelp.com/v3/businesses/secret-bao-santa-barbara/reviews?limit=20&sort_by=yelp_sort"

headers = {
    "accept": "application/json",
    "Authorization": f'Bearer {api_key}'
}

response = requests.get(url, headers=headers)

In [ ]:
print(response.status_code)

404
